In [6]:
# Memanggil library yang dibutuhkan
import ee
import geemap

# Autentikasi dan Inisialisasi
ee.Authenticate()
ee.Initialize()

# Membuat peta interaktif
Map = geemap.Map()

# ======================================================================
# DEFINISIKAN AOI PROVINSI GORONTALO
# ======================================================================
# Memuat dataset batas administrasi dan memfilter untuk Gorontalo
admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level1")
aoi_provinsi = admin_boundaries.filter(ee.Filter.eq('ADM1_NAME', 'Gorontalo'))

# Menampilkan AOI di peta
Map.addLayer(aoi_provinsi, {'color': 'blue'}, 'Batas Provinsi Gorontalo')
Map.centerObject(aoi_provinsi, 9)
Map

Map(center=[0.684882065189937, 122.38417668837405], controls=(WidgetControl(options=['position', 'transparent_…

In [9]:
# Tentukan tahun analisis
YEAR = 2020

# Memuat dan memproses data Dynamic World untuk Provinsi Gorontalo
dw_collection = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
dw_filtered = dw_collection.filterBounds(aoi_provinsi).filterDate(f'{YEAR}-01-01', f'{YEAR}-12-31')
dw_composite = dw_filtered.select('label').mode()

# Reklasifikasi 9 kelas DW ke skema Anda
reclass_dictionary = {
    'from': [0, 1, 2, 3, 4, 5, 6, 7],
    'to':   [6, 1, 2, 3, 4, 2, 6, 2]
}
lc_provinsi_2020 = dw_composite.remap(reclass_dictionary['from'], reclass_dictionary['to'])

# Menampilkan Peta Tutupan Lahan Provinsi Gorontalo
final_vis_params = {
  'min': 1, 'max': 8,
  'palette': ['#228B22', '#FFFF00', '#9370DB', '#FFA500', '#FFFF00', '#FF0000', '#9370DB', '#228B22']
}
Map.addLayer(lc_provinsi_2020, final_vis_params, f'Tutupan Lahan Provinsi {YEAR}')

print(f"Peta tutupan lahan untuk seluruh Provinsi Gorontalo tahun {YEAR} berhasil dibuat.")

Peta tutupan lahan untuk seluruh Provinsi Gorontalo tahun 2020 berhasil dibuat.


In [10]:
# --- GANTI DENGAN PATH ASET GEE BATAS DAS ANDA (Bukan tutupan lahan) ---
path_aset_das = "users/ivan_taslim/DAS_Paguyaman" # CONTOH NAMA

# Memuat batas DAS
aoi_das = ee.FeatureCollection(path_aset_das)

# Memotong peta tutupan lahan provinsi menggunakan batas DAS
lc_das_2020_final = lc_provinsi_2020.clip(aoi_das)

# Menampilkan hasil akhir yang sudah terpotong
Map.addLayer(lc_das_2020_final, final_vis_params, f'Tutupan Lahan DAS Paguyaman {YEAR}')
Map.centerObject(aoi_das, 11)

print(f"Peta tutupan lahan Provinsi Gorontalo berhasil dipotong sesuai batas DAS Paguyaman.")

Peta tutupan lahan Provinsi Gorontalo berhasil dipotong sesuai batas DAS Paguyaman.


In [12]:
# ======================================================================
# LANGKAH 6 (REVISI FINAL): EKSPOR PETA TAHUN 2020
# ======================================================================

# Variabel gambar yang benar untuk diekspor adalah hasil dari Langkah 3
# Kita tidak menggunakan '..._filled' karena tidak ada proses penambalan
gambar_final_untuk_ekspor = lc_das_2020_final 

# Proses ekspor ke GEE Assets
task = ee.batch.Export.image.toAsset(
  image=gambar_final_untuk_ekspor.toInt(),
  description=f'LC_Paguyaman_{YEAR}_DW_Generated', # Nama Task
  assetId=f'users/ivan_taslim/LC_Paguyaman_{YEAR}_DW_Generated', # Path aset tujuan Anda
  region=aoi_das.geometry(),
  scale=30,
  maxPixels=1e12
)
task.start()

print(f"Proses ekspor 'LC_Paguyaman_{YEAR}_DW_Generated' telah dimulai.")
print("PERIKSA TAB 'TASKS' DI GOOGLE EARTH ENGINE CODE EDITOR UNTUK MENJALANKANNYA.")

Proses ekspor 'LC_Paguyaman_2020_DW_Generated' telah dimulai.
PERIKSA TAB 'TASKS' DI GOOGLE EARTH ENGINE CODE EDITOR UNTUK MENJALANKANNYA.


In [17]:
# ======================================================================
# EKSPOR PETA TAHUN 2020 KE GOOGLE DRIVE
# ======================================================================

# Variabel gambar yang benar untuk diekspor adalah hasil dari Langkah 3
gambar_final_untuk_ekspor = lc_das_2020_final

# Proses ekspor ke GOOGLE DRIVE
task_drive = ee.batch.Export.image.toDrive(
  image=gambar_final_untuk_ekspor.toInt(),
  description='LC_Paguyaman_2020_DW_to_Drive', # Nama Task
  folder='Hasil_Analisis_DAS_Paguyaman', # Nama folder tujuan di Google Drive
  fileNamePrefix='LC_Paguyaman_2020_DW_Generated', # Nama file GeoTIFF
  region=aoi_das.geometry(),
  scale=30,
  maxPixels=1e12
)
task_drive.start()

print("Proses ekspor ke GOOGLE DRIVE telah dimulai.")
print("PERIKSA TAB 'TASKS' DI GOOGLE EARTH ENGINE CODE EDITOR UNTUK MENJALANKANNYA.")

Proses ekspor ke GOOGLE DRIVE telah dimulai.
PERIKSA TAB 'TASKS' DI GOOGLE EARTH ENGINE CODE EDITOR UNTUK MENJALANKANNYA.
